# Tweets about eating disorders
## 03. Training BERT Models
 
## Training and Evaluation of BERT models

The training and evaluation of 7 different BERT models on the 4 existing categorizations in the dataset is shown below:

- Category 1: in this category, tweets that have been written by people suffering from ED are represented with a value of 1 and the rest with a value of 0. In order to be able to assess this, each user profile was accessed by looking at the user's description and different tweets published by the user. In this way it was possible to determine which tweets were written by people who publicly mentioned having an ED and which were not.
- Category 2: tweets that promoted having an ED were labelled with a value of 1 and all other tweets were labelled with a value of 0. There are communities of people who suffer from EDs who try to encourage other people to also suffer from it by promoting it as if it were something positive or fashionable. There are many studies that talk about pro ED communities and that are detected with terms such as "proana" or "pro-anorexia" 9.
- Category 3: in this category, informative tweets were represented with a value of 1 and non-informative tweets with a value of 0. Informative tweets are those that show information with the aim of informing readers, while the rest are written texts in which the author reflects a subjective opinion.
- Category 4: in category 4, scientific tweets were labelled with a value of 1 and the rest with a value of 0. A tweet of an informative nature that had been written by a person belonging to the field of research, for example, a doctor of philosophy in different subjects, was labelled as a scientific tweet. Scientific tweets were also those that shared links to articles published in scientific journals.

The 7 BERT pre-trained models used are as follows:

- RoBERTa 
- BERT 
- CamemBERT 
- DistilBERT 
- FlauBERT 
- ALBERT 
- RobBERT

The selected parameters can be seen in the train_args variable in the next cell. The selected parameters were the ones that gave the best results after several previous tests.

In [3]:
import os
import pandas as pd
import numpy as np
import json
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re, string, unicodedata
import nltk
from nltk import word_tokenize, sent_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
nltk.download
tweets = pd.read_csv('tweets_cleaned.csv', encoding='utf8', error_bad_lines=False)

In [4]:
tweets.columns=['num1','stream_group','text_orig','f1_commercial','f2_politics','f3_ed','f4_family','f5_edpatient','f6_proed','f7_offensive','f8_info','f9_scientific','f10_sad','hashtag','text','segmented','text_similar']

In [5]:
cols=['f1_commercial','f2_politics','f3_ed','f4_family','f5_edpatient','f6_proed','f7_offensive','f8_info','f9_scientific','f10_sad']

In [6]:
tweets1 = tweets.copy().drop(['f1_commercial', 'f2_politics', 'f3_ed', 'f4_family', 'f7_offensive', 'f10_sad'], axis=1)

In [7]:
tweets1.head(2)

,num1,stream_group,text_orig,f5_edpatient,f6_proed,f8_info,f9_scientific,hashtag,text,segmented,text_similar
0,0,1,RT @beatED: Learn more about anorexia and buli...,0,0,1,0,['BBCPanorama'],"['learn', 'anorexia', 'bulimia', 'well', 'eati...",bbc panorama,[]
1,1,1,A woman tries to balance her relationships wit...,0,0,0,0,"['anorexia', 'BodyofWater']","['woman', 'try', 'balance', 'relationship', 'm...",anorexia bodyof water,[]


In [8]:
import pandas as pd
import numpy as np

import spacy
import nltk
import nltk.data
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import regex as re
import string
from collections import defaultdict

import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_colwidth', None)

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

from simpletransformers.classification import ClassificationModel


import io

C:\Users\MICROSOFT\Anaconda3\envs\ED\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: [WinError 126] No se puede encontrar el módulo especificado
  warn(f"Failed to load image Python extension: {e}")


In [9]:
punctuations = "¡!#$%&'()*+,-./:;<=>¿?@[\]^_`{|}~"

def read_txt(filename):
    list = []
    with open(filename, 'r', encoding='utf-8') as f:
        data = f.readlines()
        for line in data:
            list.append(str(line).replace('\n', ''))
    return list

stopwords = read_txt('english_stopwords.txt')

stemmer = SnowballStemmer('english')


def clean_accents(tweet):
    tweet = re.sub(r"[àáâãäå]", "a", tweet)
    tweet = re.sub(r"ç", "c", tweet)
    tweet = re.sub(r"[èéêë]", "e", tweet)
    tweet = re.sub(r"[ìíîï]", "i", tweet)
    tweet = re.sub(r"[òóôõö]", "o", tweet)
    tweet = re.sub(r"[ùúûü]", "u", tweet)
    tweet = re.sub(r"[ýÿ]", "y", tweet)

    return tweet

def clean_tweet(tweet, stem = False):
    tweet = tweet.lower().strip()
    tweet = re.sub(r'https?:\/\/\S+', '', tweet)
    tweet = re.sub(r'http?:\/\/\S+', '', tweet)
    tweet = re.sub(r'www?:\/\/\S+', '', tweet)
    tweet = re.sub(r'\s([@#][\w_-]+)', "", tweet)
    tweet = re.sub(r"\n", " ", tweet)
    tweet = clean_accents(tweet)
    tweet = re.sub(r"\b(a*ha+h[ha]*|o?l+o+l+[ol]*|x+d+[x*d*]*|a*ja+[j+a+]+)\b", "<risas>", tweet)
    for symbol in punctuations:
        tweet = tweet.replace(symbol, "")
    tokens = []
    for token in tweet.strip().split():
        if token not in punctuations and token not in stopwords:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

In [10]:
tweets1['text_cleaned'] = tweets['text_orig'].apply(lambda s : clean_tweet(s))
print(tweets1['text_cleaned'].head(5))

0                                                                   rt learn anorexia bulimia well eating disorders
1      woman tries balance relationships mother teenage daughter shadow atmospheric british drama depot friday book
2                           full diagnosis like therapist legit told mom anorexia nevrususu poor mom sat like 👁👄👁oh
3                                          highercalorie diets patients anorexia nervosa shorten hospital stays via
4    tw ed ment freddie pissed amazing way got anorexic bulimic actress play foodobsessed character never sit right
Name: text_cleaned, dtype: object


In [11]:
tweets1.head(2)

,num1,stream_group,text_orig,f5_edpatient,f6_proed,f8_info,f9_scientific,hashtag,text,segmented,text_similar,text_cleaned
0,0,1,"RT @beatED: Learn more about anorexia and bulimia, as well as other eating disorders, here: https://t.co/Aj2HbjRH39 @BBCPanorama #BBCPanorama",0,0,1,0,['BBCPanorama'],"['learn', 'anorexia', 'bulimia', 'well', 'eating', 'disorder']",bbc panorama,[],rt learn anorexia bulimia well eating disorders
1,1,1,"A woman tries to balance her relationships with her mother and teenage daughter while under the shadow of #anorexia in the atmospheric British drama #BodyofWater. \n\nAt Depot from Friday, book now: https://lewesdepot.org/film/body-of-water\n",0,0,0,0,"['anorexia', 'BodyofWater']","['woman', 'try', 'balance', 'relationship', 'mother', 'teenage', 'daughter', 'shadow', 'atmospheric', 'british', 'drama', 'depot', 'friday', 'book']",anorexia bodyof water,[],woman tries balance relationships mother teenage daughter shadow atmospheric british drama depot friday book


In [10]:
df = tweets1.copy()
X = df['text_cleaned']
Y1 = df['f5_edpatient']
Y2 = df['f6_proed']
Y3 = df['f8_info']
Y4 = df['f9_scientific']

In [11]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X, Y1, test_size=0.3, random_state=42)
X2_train, X2_test, y2_train, y2_test = train_test_split(X, Y2, test_size=0.3, random_state=42)
X3_train, X3_test, y3_train, y3_test = train_test_split(X, Y3, test_size=0.3, random_state=42)
X4_train, X4_test, y4_train, y4_test = train_test_split(X, Y4, test_size=0.3, random_state=42)

In [12]:
y1_test.value_counts(normalize=True)

1    0.51773
0    0.48227
Name: f5_edpatient, dtype: float64

In [13]:
train_args ={"reprocess_input_data": True,
           "fp16":False,
             "evaluate_during_training": False,
             "evaluate_during_training_verbose":False,
             "learning_rate":2e-5,
             "train_batch_size":32,
             "eval_batch_size":32,
           "num_train_epochs": 15, 'overwrite_output_dir': True, "evaluation_strategy":'epochs'
            }

def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')

def calcule_f1(df):
    return(df['tp'] / (df['tp'] + 0.5 * (df['fp'] + df['fn'])))

import torch
import gc
from tqdm import tqdm


dfEval1 = pd.DataFrame()

In [14]:
indexBERT = ['BERT','RoBERTa','DistilBERT','CamemBERT','Albert','FlauBERT','RobBERT']

train_df1 = pd.DataFrame({ 'text_cleaned': X1_train, 'target': y1_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X1_test, 'target': y1_test })

train_df2 = pd.DataFrame({ 'text_cleaned': X2_train, 'target': y2_train })
test_df2 = pd.DataFrame({ 'text_cleaned': X2_test, 'target': y2_test })

train_df3 = pd.DataFrame({ 'text_cleaned': X3_train, 'target': y3_train })
test_df3 = pd.DataFrame({ 'text_cleaned': X3_test, 'target': y3_test })

train_df4 = pd.DataFrame({ 'text_cleaned': X4_train, 'target': y4_train })
test_df4 = pd.DataFrame({ 'text_cleaned': X4_test, 'target': y4_test })


In [15]:
N_ITER = 5

In [16]:
gc.collect()
torch.cuda.empty_cache()

In [17]:
'''
torch.load(model1, 'model1.pt')
model1.train_model(train_df1)
result1, model_outputs1, wrong_predictions1 = model1.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
print(result1)
del model1
gc.collect()
'''
#gc.collect()
#torch.cuda.empty_cache()
limitsave=0
train_df1 = pd.DataFrame({ 'text_cleaned': X1_train, 'target': y1_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X1_test, 'target': y1_test })

'''
train_df1 = pd.DataFrame({ 'text_cleaned': X2_train, 'target': y2_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X2_test, 'target': y2_test })

train_df1 = pd.DataFrame({ 'text_cleaned': X3_train, 'target': y3_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X3_test, 'target': y3_test })

train_df1 = pd.DataFrame({ 'text_cleaned': X4_train, 'target': y4_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X4_test, 'target': y4_test })
'''


for i in range(0,N_ITER):
    model1 = ClassificationModel(
    "bert", "bert-base-multilingual-cased",
        use_cuda = True,
        args=train_args
    )
    model1.train_model(train_df1)
    result1, model_outputs1, wrong_predictions1 = model1.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1)
    if(i<limitsave):
        torch.save(model1, 'model1'+str(i)+'.pt')
    del model1
    gc.collect()
    torch.cuda.empty_cache()
    if(i==0):
        dfResultsModels1 = pd.DataFrame.from_dict(result1, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1, orient="index").T
        dfResultsModels1 = dfResultsModels1.append(dfResultsModels1b)
        
dfResultsModels1Trans = pd.DataFrame(dfResultsModels1.mean(axis=0)).T
dfResultsModels1Trans['f1'] = calcule_f1(dfResultsModels1Trans)

dfResultsModelsTotal = dfResultsModels1Trans.copy()
dfResultsModelsTotal.to_csv('dfResultsModelsTotal-1.csv')

#dfResultsModelsTotal = pd.read_csv('dfResultsModelsTotal.csv', encoding='utf8', error_bad_lines=False)
        
for i in range(0,N_ITER):
    model1c = ClassificationModel(
    "roberta", "roberta-base",
        use_cuda = True,
        args=train_args
    )
    model1c.train_model(train_df1)

    result1c, model_outputs1c, wrong_predictions1c = model1c.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1c)
    if(i<limitsave):
        torch.save(model1c, 'model1c'+str(i)+'.pt')
    del model1c
    gc.collect()
    if(i==0):
        dfResultsModels1c = pd.DataFrame.from_dict(result1c, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1c, orient="index").T
        dfResultsModels1c = dfResultsModels1c.append(dfResultsModels1b)

dfResultsModels1cTrans = pd.DataFrame(dfResultsModels1c.mean(axis=0)).T
dfResultsModels1cTrans['f1'] = calcule_f1(dfResultsModels1cTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1cTrans)
dfResultsModelsTotal.to_csv('dfResultsModelsTotal-1.csv')



for i in range(0,N_ITER):
    model1d = ClassificationModel(
    "distilbert", "distilbert-base-cased",
        use_cuda = True,
        args=train_args
    )
    model1d.train_model(train_df1)

    result1d, model_outputs1d, wrong_predictions1d = model1d.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1d)
    if(i<limitsave):
        torch.save(model1d, 'model1d'+str(i)+'.pt')
    del model1d
    gc.collect()
    if(i==0):
        dfResultsModels1d = pd.DataFrame.from_dict(result1d, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1d, orient="index").T
        dfResultsModels1d = dfResultsModels1d.append(dfResultsModels1b)

dfResultsModels1dTrans = pd.DataFrame(dfResultsModels1d.mean(axis=0)).T
dfResultsModels1dTrans['f1'] = calcule_f1(dfResultsModels1dTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1dTrans)

dfResultsModelsTotal.to_csv('dfResultsModelsTotal-1.csv')



for i in range(0,N_ITER):
    model1e = ClassificationModel(
    "camembert", "camembert-base",
        use_cuda = True,
        args=train_args
    )
    model1e.train_model(train_df1)

    result1e, model_outputs1e, wrong_predictions1e = model1e.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1e)
    if(i<limitsave):
        torch.save(model1e, 'model1e'+str(i)+'.pt')
    del model1e
    gc.collect()
    if(i==0):
        dfResultsModels1e = pd.DataFrame.from_dict(result1e, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1e, orient="index").T
        dfResultsModels1e = dfResultsModels1e.append(dfResultsModels1b)

dfResultsModels1eTrans = pd.DataFrame(dfResultsModels1e.mean(axis=0)).T
dfResultsModels1eTrans['f1'] = calcule_f1(dfResultsModels1eTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1eTrans)

dfResultsModelsTotal.to_csv('dfResultsModelsTotal-1.csv')


for i in range(0,N_ITER):
    model1f = ClassificationModel(
    "albert", "albert-base-v1",
        use_cuda = True,
        args=train_args
    )
    model1f.train_model(train_df1)

    result1f, model_outputs1f, wrong_predictions1f = model1f.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1f)
    if(i<limitsave):
        torch.save(model1f, 'model1f'+str(i)+'.pt')
    del model1f
    gc.collect()
    if(i==0):
        dfResultsModels1f = pd.DataFrame.from_dict(result1f, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1f, orient="index").T
        dfResultsModels1f = dfResultsModels1f.append(dfResultsModels1b)

dfResultsModels1fTrans = pd.DataFrame(dfResultsModels1f.mean(axis=0)).T
dfResultsModels1fTrans['f1'] = calcule_f1(dfResultsModels1fTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1fTrans)
dfResultsModelsTotal.to_csv('dfResultsModelsTotal-1.csv')



for i in range(0,N_ITER):
    model1g = ClassificationModel(
    "flaubert","flaubert/flaubert_base_cased",
        use_cuda = True,
        args=train_args
    )
    model1g.train_model(train_df1)

    result1g, model_outputs1g, wrong_predictions1g = model1g.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1g)
    if(i<limitsave):
        torch.save(model1g, 'model1g'+str(i)+'.pt')
    del model1g
    gc.collect()
    if(i==0):
        dfResultsModels1g = pd.DataFrame.from_dict(result1g, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1g, orient="index").T
        dfResultsModels1g = dfResultsModels1g.append(dfResultsModels1b)

dfResultsModels1gTrans = pd.DataFrame(dfResultsModels1g.mean(axis=0)).T
dfResultsModels1gTrans['f1'] = calcule_f1(dfResultsModels1gTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1gTrans)
dfResultsModelsTotal.to_csv('dfResultsModelsTotal-1.csv')


for i in range(0,N_ITER):
    model1g = ClassificationModel(
    "robbert","pdelobelle/robbert-v2-dutch-base",
        use_cuda = True,
        args=train_args
    )
    model1g.train_model(train_df1)

    result1g, model_outputs1g, wrong_predictions1g = model1g.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1g)
    if(i<limitsave):
        torch.save(model1g, 'model1g'+str(i)+'.pt')
    del model1g
    gc.collect()
    if(i==0):
        dfResultsModels1g = pd.DataFrame.from_dict(result1g, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1g, orient="index").T
        dfResultsModels1g = dfResultsModels1g.append(dfResultsModels1b)

dfResultsModels1gTrans = pd.DataFrame(dfResultsModels1g.mean(axis=0)).T
dfResultsModels1gTrans['f1'] = calcule_f1(dfResultsModels1gTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1gTrans)
dfResultsModelsTotal.to_csv('dfResultsModelsTotal-1.csv')

for i in range(0,N_ITER):
    model1h = ClassificationModel(
    "roberta","pdelobelle/robbert-v2-dutch-base",
        use_cuda = True,
        args=train_args
    )
    model1h.train_model(train_df1)

    result1h, model_outputs1h, wrong_predictions1h = model1h.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1h)
    if(i<limitsave):
        torch.save(model1h, 'model1h'+str(i)+'.pt')
    del model1h
    gc.collect()
    if(i==0):
        dfResultsModels1h = pd.DataFrame.from_dict(result1h, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1h, orient="index").T
        dfResultsModels1h = dfResultsModels1h.append(dfResultsModels1b)
        
dfResultsModels1hTrans = pd.DataFrame(dfResultsModels1h.mean(axis=0)).T
dfResultsModels1hTrans['f1'] = calcule_f1(dfResultsModels1hTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1hTrans)
dfResultsModelsTotal.to_csv('dfResultsModelsTotal-1.csv')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

  0%|          | 0/1313 [00:00<?, ?it/s]

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

...

C:\Users\MICROSOFT\Anaconda3\envs\ED\lib\site-packages\simpletransformers\classification\classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/564 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/18 [00:00<?, ?it/s]

{'mcc': 0.6131694033219272, 'tp': 236, 'tn': 219, 'fp': 53, 'fn': 56, 'auroc': 0.8889126712328768, 'auprc': 0.8880633061782093, 'f1': 0.8067375886524821, 'acc': 0.8067375886524822, 'eval_loss': 0.7385811474588182}


KeyError: 'robbert'

In [19]:
for i in range(0,N_ITER):
    model1h = ClassificationModel(
    "roberta","pdelobelle/robbert-v2-dutch-base",
        use_cuda = True,
        args=train_args
    )
    model1h.train_model(train_df1)

    result1h, model_outputs1h, wrong_predictions1h = model1h.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1h)
    if(i<limitsave):
        torch.save(model1h, 'model1h'+str(i)+'.pt')
    del model1h
    gc.collect()
    if(i==0):
        dfResultsModels1h = pd.DataFrame.from_dict(result1h, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1h, orient="index").T
        dfResultsModels1h = dfResultsModels1h.append(dfResultsModels1b)
        
dfResultsModels1hTrans = pd.DataFrame(dfResultsModels1h.mean(axis=0)).T
dfResultsModels1hTrans['f1'] = calcule_f1(dfResultsModels1hTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1hTrans)
dfResultsModelsTotal.to_csv('dfResultsModelsTotal-1.csv')

Some weights of the model checkpoint at pdelobelle/robbert-v2-dutch-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base and are newly initialized: ['classifier.out_proj.bias', 'clas

  0%|          | 0/1313 [00:00<?, ?it/s]

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

...

Running Epoch 14 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

C:\Users\MICROSOFT\Anaconda3\envs\ED\lib\site-packages\simpletransformers\classification\classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/564 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/18 [00:00<?, ?it/s]

{'mcc': 0.5465272524254265, 'tp': 213, 'tn': 222, 'fp': 50, 'fn': 79, 'auroc': 0.8710213537469782, 'auprc': 0.8651848845286932, 'f1': 0.7712765957446808, 'acc': 0.7712765957446809, 'eval_loss': 1.014398214717706}


In [20]:
indexBERT = ['BERT','RoBERTa','DistilBERT','CamemBERT','Albert','Flaubert','RobBERT']

#dfResultsModelsTotal.reindex(indexBERT)
dfResultsModelsTotal = dfResultsModelsTotal.reset_index(drop=True)
dfResultsModelsTotal.index = indexBERT
dfResultsModelsTotal

dfResultssModelsTotal1 = dfResultsModelsTotal.copy()

dfResultssModelsTotal1

,mcc,tp,tn,fp,fn,auroc,auprc,f1,acc,eval_loss
BERT,0.661133,239.0,229.2,42.8,53.0,0.889852,0.872338,0.833043,0.830142,1.098579
RoBERTa,0.661664,246.8,221.8,50.2,45.2,0.893168,0.881878,0.838031,0.830851,1.093904
DistilBERT,0.661993,249.4,219.4,52.6,42.6,0.884118,0.861300,0.839731,0.831206,0.998397
CamemBERT,0.574287,227.2,216.6,55.4,64.8,0.867355,0.852618,0.790811,0.786879,0.738183
Albert,0.606882,238.8,214.4,57.6,53.2,0.873935,0.859981,0.811693,0.803546,1.001325
Flaubert,0.634536,244.0,217.0,55.0,48.0,0.893810,0.892281,0.825719,0.817376,0.745874
RobBERT,0.569540,226.2,216.2,55.8,65.8,0.871104,0.852964,0.788153,0.784397,1.086767


In [21]:
'''
torch.load(model1, 'model1.pt')
model1.train_model(train_df1)
result1, model_outputs1, wrong_predictions1 = model1.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
print(result1)
del model1
gc.collect()
'''
#gc.collect()
#torch.cuda.empty_cache()
limitsave=0

'''
train_df1 = pd.DataFrame({ 'text_cleaned': X1_train, 'target': y1_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X1_test, 'target': y1_test })
'''

train_df1 = pd.DataFrame({ 'text_cleaned': X2_train, 'target': y2_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X2_test, 'target': y2_test })
'''
train_df1 = pd.DataFrame({ 'text_cleaned': X3_train, 'target': y3_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X3_test, 'target': y3_test })

train_df1 = pd.DataFrame({ 'text_cleaned': X4_train, 'target': y4_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X4_test, 'target': y4_test })
'''

for i in range(0,N_ITER):
    model1 = ClassificationModel(
    "bert", "bert-base-multilingual-cased",
        use_cuda = True,
        args=train_args
    )
    model1.train_model(train_df1)
    result1, model_outputs1, wrong_predictions1 = model1.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1)
    if(i<limitsave):
        torch.save(model1, 'model1'+str(i)+'.pt')
    del model1
    gc.collect()
    torch.cuda.empty_cache()
    if(i==0):
        dfResultsModels1 = pd.DataFrame.from_dict(result1, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1, orient="index").T
        dfResultsModels1 = dfResultsModels1.append(dfResultsModels1b)
        
dfResultsModels1Trans = pd.DataFrame(dfResultsModels1.mean(axis=0)).T
dfResultsModels1Trans['f1'] = calcule_f1(dfResultsModels1Trans)

dfResultsModelsTotal = dfResultsModels1Trans.copy()
dfResultsModelsTotal.to_csv('dfResultsModelsTotal-2.csv')

#dfResultsModelsTotal = pd.read_csv('dfResultsModelsTotal.csv', encoding='utf8', error_bad_lines=False)
        
for i in range(0,N_ITER):
    model1c = ClassificationModel(
    "roberta", "roberta-base",
        use_cuda = True,
        args=train_args
    )
    model1c.train_model(train_df1)

    result1c, model_outputs1c, wrong_predictions1c = model1c.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1c)
    if(i<limitsave):
        torch.save(model1c, 'model1c'+str(i)+'.pt')
    del model1c
    gc.collect()
    if(i==0):
        dfResultsModels1c = pd.DataFrame.from_dict(result1c, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1c, orient="index").T
        dfResultsModels1c = dfResultsModels1c.append(dfResultsModels1b)

dfResultsModels1cTrans = pd.DataFrame(dfResultsModels1c.mean(axis=0)).T
dfResultsModels1cTrans['f1'] = calcule_f1(dfResultsModels1cTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1cTrans)
dfResultsModelsTotal.to_csv('dfResultsModelsTotal-2.csv')



for i in range(0,N_ITER):
    model1d = ClassificationModel(
    "distilbert", "distilbert-base-cased",
        use_cuda = True,
        args=train_args
    )
    model1d.train_model(train_df1)

    result1d, model_outputs1d, wrong_predictions1d = model1d.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1d)
    if(i<limitsave):
        torch.save(model1d, 'model1d'+str(i)+'.pt')
    del model1d
    gc.collect()
    if(i==0):
        dfResultsModels1d = pd.DataFrame.from_dict(result1d, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1d, orient="index").T
        dfResultsModels1d = dfResultsModels1d.append(dfResultsModels1b)

dfResultsModels1dTrans = pd.DataFrame(dfResultsModels1d.mean(axis=0)).T
dfResultsModels1dTrans['f1'] = calcule_f1(dfResultsModels1dTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1dTrans)

dfResultsModelsTotal.to_csv('dfResultsModelsTotal-2.csv')



for i in range(0,N_ITER):
    model1e = ClassificationModel(
    "camembert", "camembert-base",
        use_cuda = True,
        args=train_args
    )
    model1e.train_model(train_df1)

    result1e, model_outputs1e, wrong_predictions1e = model1e.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1e)
    if(i<limitsave):
        torch.save(model1e, 'model1e'+str(i)+'.pt')
    del model1e
    gc.collect()
    if(i==0):
        dfResultsModels1e = pd.DataFrame.from_dict(result1e, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1e, orient="index").T
        dfResultsModels1e = dfResultsModels1e.append(dfResultsModels1b)

dfResultsModels1eTrans = pd.DataFrame(dfResultsModels1e.mean(axis=0)).T
dfResultsModels1eTrans['f1'] = calcule_f1(dfResultsModels1eTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1eTrans)

dfResultsModelsTotal.to_csv('dfResultsModelsTotal-2.csv')


for i in range(0,N_ITER):
    model1f = ClassificationModel(
    "albert", "albert-base-v1",
        use_cuda = True,
        args=train_args
    )
    model1f.train_model(train_df1)

    result1f, model_outputs1f, wrong_predictions1f = model1f.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1f)
    if(i<limitsave):
        torch.save(model1f, 'model1f'+str(i)+'.pt')
    del model1f
    gc.collect()
    if(i==0):
        dfResultsModels1f = pd.DataFrame.from_dict(result1f, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1f, orient="index").T
        dfResultsModels1f = dfResultsModels1f.append(dfResultsModels1b)

dfResultsModels1fTrans = pd.DataFrame(dfResultsModels1f.mean(axis=0)).T
dfResultsModels1fTrans['f1'] = calcule_f1(dfResultsModels1fTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1fTrans)
dfResultsModelsTotal.to_csv('dfResultsModelsTotal-2.csv')



for i in range(0,N_ITER):
    model1g = ClassificationModel(
    "flaubert","flaubert/flaubert_base_cased",
        use_cuda = True,
        args=train_args
    )
    model1g.train_model(train_df1)

    result1g, model_outputs1g, wrong_predictions1g = model1g.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1g)
    if(i<limitsave):
        torch.save(model1g, 'model1g'+str(i)+'.pt')
    del model1g
    gc.collect()
    if(i==0):
        dfResultsModels1g = pd.DataFrame.from_dict(result1g, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1g, orient="index").T
        dfResultsModels1g = dfResultsModels1g.append(dfResultsModels1b)

dfResultsModels1gTrans = pd.DataFrame(dfResultsModels1g.mean(axis=0)).T
dfResultsModels1gTrans['f1'] = calcule_f1(dfResultsModels1gTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1gTrans)
dfResultsModelsTotal.to_csv('dfResultsModelsTotal-2.csv')

        
for i in range(0,N_ITER):
    model1h = ClassificationModel(
    "roberta","pdelobelle/robbert-v2-dutch-base",
        use_cuda = True,
        args=train_args
    )
    model1h.train_model(train_df1)

    result1h, model_outputs1h, wrong_predictions1h = model1h.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1h)
    if(i<limitsave):
        torch.save(model1h, 'model1h'+str(i)+'.pt')
    del model1h
    gc.collect()
    if(i==0):
        dfResultsModels1h = pd.DataFrame.from_dict(result1h, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1h, orient="index").T
        dfResultsModels1h = dfResultsModels1h.append(dfResultsModels1b)
        
dfResultsModels1hTrans = pd.DataFrame(dfResultsModels1h.mean(axis=0)).T
dfResultsModels1hTrans['f1'] = calcule_f1(dfResultsModels1hTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1hTrans)
dfResultsModelsTotal.to_csv('dfResultsModelsTotal-2.csv')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

  0%|          | 0/1313 [00:00<?, ?it/s]

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

...

Running Epoch 14 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

C:\Users\MICROSOFT\Anaconda3\envs\ED\lib\site-packages\simpletransformers\classification\classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/564 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/18 [00:00<?, ?it/s]

{'mcc': 0.6300854921702378, 'tp': 97, 'tn': 391, 'fp': 36, 'fn': 40, 'auroc': 0.9084257850561548, 'auprc': 0.767427501619759, 'f1': 0.8652482269503546, 'acc': 0.8652482269503546, 'eval_loss': 0.7958763663967451}


In [22]:

#dfResultsModelsTotal.reindex(indexBERT)
dfResultsModelsTotal = dfResultsModelsTotal.reset_index(drop=True)
dfResultsModelsTotal.index = indexBERT
dfResultsModelsTotal

dfResultssModelsTotal2 = dfResultsModelsTotal.copy()

dfResultssModelsTotal2

,mcc,tp,tn,fp,fn,auroc,auprc,f1,acc,eval_loss
BERT,0.638673,92.6,399.0,28.0,44.4,0.928467,0.814803,0.718944,0.871631,0.886287
RoBERTa,0.674004,93.6,405.6,21.4,43.4,0.931117,0.826344,0.742857,0.885106,0.823541
DistilBERT,0.643608,93.0,399.6,27.4,44.0,0.921417,0.806144,0.722611,0.873404,0.701475
CamemBERT,0.658367,96.4,398.6,28.4,40.6,0.920934,0.789542,0.736440,0.877660,0.443009
Albert,0.667905,96.4,400.8,26.2,40.6,0.920976,0.802526,0.742681,0.881560,0.640400
Flaubert,0.649517,95.4,397.8,29.2,41.6,0.919458,0.782713,0.729358,0.874468,0.723149
RobBERT,0.620400,96.0,390.0,37.0,41.0,0.915981,0.778671,0.711111,0.861702,0.793253


In [23]:
'''
torch.load(model1, 'model1.pt')
model1.train_model(train_df1)
result1, model_outputs1, wrong_predictions1 = model1.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
print(result1)
del model1
gc.collect()
'''
#gc.collect()
#torch.cuda.empty_cache()
limitsave=0

'''
train_df1 = pd.DataFrame({ 'text_cleaned': X1_train, 'target': y1_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X1_test, 'target': y1_test })


train_df1 = pd.DataFrame({ 'text_cleaned': X2_train, 'target': y2_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X2_test, 'target': y2_test })
'''
train_df1 = pd.DataFrame({ 'text_cleaned': X3_train, 'target': y3_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X3_test, 'target': y3_test })
'''
train_df1 = pd.DataFrame({ 'text_cleaned': X4_train, 'target': y4_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X4_test, 'target': y4_test })
'''

for i in range(0,N_ITER):
    model1 = ClassificationModel(
    "bert", "bert-base-multilingual-cased",
        use_cuda = True,
        args=train_args
    )
    model1.train_model(train_df1)
    result1, model_outputs1, wrong_predictions1 = model1.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1)
    if(i<limitsave):
        torch.save(model1, 'model1'+str(i)+'.pt')
    del model1
    gc.collect()
    torch.cuda.empty_cache()
    if(i==0):
        dfResultsModels1 = pd.DataFrame.from_dict(result1, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1, orient="index").T
        dfResultsModels1 = dfResultsModels1.append(dfResultsModels1b)
        
dfResultsModels1Trans = pd.DataFrame(dfResultsModels1.mean(axis=0)).T
dfResultsModels1Trans['f1'] = calcule_f1(dfResultsModels1Trans)

dfResultsModelsTotal = dfResultsModels1Trans.copy()
dfResultsModelsTotal.to_csv('dfResultsModelsTotal-3.csv')

#dfResultsModelsTotal = pd.read_csv('dfResultsModelsTotal.csv', encoding='utf8', error_bad_lines=False)
        
for i in range(0,N_ITER):
    model1c = ClassificationModel(
    "roberta", "roberta-base",
        use_cuda = True,
        args=train_args
    )
    model1c.train_model(train_df1)

    result1c, model_outputs1c, wrong_predictions1c = model1c.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1c)
    if(i<limitsave):
        torch.save(model1c, 'model1c'+str(i)+'.pt')
    del model1c
    gc.collect()
    if(i==0):
        dfResultsModels1c = pd.DataFrame.from_dict(result1c, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1c, orient="index").T
        dfResultsModels1c = dfResultsModels1c.append(dfResultsModels1b)

dfResultsModels1cTrans = pd.DataFrame(dfResultsModels1c.mean(axis=0)).T
dfResultsModels1cTrans['f1'] = calcule_f1(dfResultsModels1cTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1cTrans)
dfResultsModelsTotal.to_csv('dfResultsModelsTotal-3.csv')



for i in range(0,N_ITER):
    model1d = ClassificationModel(
    "distilbert", "distilbert-base-cased",
        use_cuda = True,
        args=train_args
    )
    model1d.train_model(train_df1)

    result1d, model_outputs1d, wrong_predictions1d = model1d.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1d)
    if(i<limitsave):
        torch.save(model1d, 'model1d'+str(i)+'.pt')
    del model1d
    gc.collect()
    if(i==0):
        dfResultsModels1d = pd.DataFrame.from_dict(result1d, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1d, orient="index").T
        dfResultsModels1d = dfResultsModels1d.append(dfResultsModels1b)

dfResultsModels1dTrans = pd.DataFrame(dfResultsModels1d.mean(axis=0)).T
dfResultsModels1dTrans['f1'] = calcule_f1(dfResultsModels1dTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1dTrans)

dfResultsModelsTotal.to_csv('dfResultsModelsTotal-3.csv')



for i in range(0,N_ITER):
    model1e = ClassificationModel(
    "camembert", "camembert-base",
        use_cuda = True,
        args=train_args
    )
    model1e.train_model(train_df1)

    result1e, model_outputs1e, wrong_predictions1e = model1e.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1e)
    if(i<limitsave):
        torch.save(model1e, 'model1e'+str(i)+'.pt')
    del model1e
    gc.collect()
    if(i==0):
        dfResultsModels1e = pd.DataFrame.from_dict(result1e, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1e, orient="index").T
        dfResultsModels1e = dfResultsModels1e.append(dfResultsModels1b)

dfResultsModels1eTrans = pd.DataFrame(dfResultsModels1e.mean(axis=0)).T
dfResultsModels1eTrans['f1'] = calcule_f1(dfResultsModels1eTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1eTrans)

dfResultsModelsTotal.to_csv('dfResultsModelsTotal-3.csv')


for i in range(0,N_ITER):
    model1f = ClassificationModel(
    "albert", "albert-base-v1",
        use_cuda = True,
        args=train_args
    )
    model1f.train_model(train_df1)

    result1f, model_outputs1f, wrong_predictions1f = model1f.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1f)
    if(i<limitsave):
        torch.save(model1f, 'model1f'+str(i)+'.pt')
    del model1f
    gc.collect()
    if(i==0):
        dfResultsModels1f = pd.DataFrame.from_dict(result1f, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1f, orient="index").T
        dfResultsModels1f = dfResultsModels1f.append(dfResultsModels1b)

dfResultsModels1fTrans = pd.DataFrame(dfResultsModels1f.mean(axis=0)).T
dfResultsModels1fTrans['f1'] = calcule_f1(dfResultsModels1fTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1fTrans)
dfResultsModelsTotal.to_csv('dfResultsModelsTotal-3.csv')



for i in range(0,N_ITER):
    model1g = ClassificationModel(
    "flaubert","flaubert/flaubert_base_cased",
        use_cuda = True,
        args=train_args
    )
    model1g.train_model(train_df1)

    result1g, model_outputs1g, wrong_predictions1g = model1g.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1g)
    if(i<limitsave):
        torch.save(model1g, 'model1g'+str(i)+'.pt')
    del model1g
    gc.collect()
    if(i==0):
        dfResultsModels1g = pd.DataFrame.from_dict(result1g, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1g, orient="index").T
        dfResultsModels1g = dfResultsModels1g.append(dfResultsModels1b)

dfResultsModels1gTrans = pd.DataFrame(dfResultsModels1g.mean(axis=0)).T
dfResultsModels1gTrans['f1'] = calcule_f1(dfResultsModels1gTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1gTrans)
dfResultsModelsTotal.to_csv('dfResultsModelsTotal-3.csv')

for i in range(0,N_ITER):
    model1h = ClassificationModel(
    "roberta","pdelobelle/robbert-v2-dutch-base",
        use_cuda = True,
        args=train_args
    )
    model1h.train_model(train_df1)

    result1h, model_outputs1h, wrong_predictions1h = model1h.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1h)
    if(i<limitsave):
        torch.save(model1h, 'model1h'+str(i)+'.pt')
    del model1h
    gc.collect()
    if(i==0):
        dfResultsModels1h = pd.DataFrame.from_dict(result1h, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1h, orient="index").T
        dfResultsModels1h = dfResultsModels1h.append(dfResultsModels1b)
        
dfResultsModels1hTrans = pd.DataFrame(dfResultsModels1h.mean(axis=0)).T
dfResultsModels1hTrans['f1'] = calcule_f1(dfResultsModels1hTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1hTrans)
dfResultsModelsTotal.to_csv('dfResultsModelsTotal-3.csv')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

  0%|          | 0/1313 [00:00<?, ?it/s]

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

...

Running Epoch 14 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

C:\Users\MICROSOFT\Anaconda3\envs\ED\lib\site-packages\simpletransformers\classification\classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/564 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/18 [00:00<?, ?it/s]

{'mcc': 0.6021965489053792, 'tp': 144, 'tn': 318, 'fp': 60, 'fn': 42, 'auroc': 0.8630454571314787, 'auprc': 0.7288629663313922, 'f1': 0.8191489361702128, 'acc': 0.8191489361702128, 'eval_loss': 1.007549472981029}


In [24]:

#dfResultsModelsTotal.reindex(indexBERT)
dfResultsModelsTotal = dfResultsModelsTotal.reset_index(drop=True)
dfResultsModelsTotal.index = indexBERT
dfResultsModelsTotal

dfResultssModelsTotal3 = dfResultsModelsTotal.copy()

dfResultssModelsTotal3

,mcc,tp,tn,fp,fn,auroc,auprc,f1,acc,eval_loss
BERT,0.657822,153.0,322.4,55.6,33.0,0.884454,0.798259,0.775469,0.842908,0.992379
RoBERTa,0.658711,152.0,324.0,54.0,34.0,0.881954,0.789412,0.775510,0.843972,0.928282
DistilBERT,0.621301,145.4,321.8,56.2,40.6,0.880053,0.755683,0.750258,0.828369,0.888164
CamemBERT,0.611891,152.0,309.0,69.0,34.0,0.885845,0.798341,0.746929,0.817376,0.611968
Albert,0.599814,147.2,312.4,65.6,38.8,0.882070,0.782156,0.738215,0.814894,0.967424
Flaubert,0.572897,146.6,304.6,73.4,39.4,0.879166,0.801887,0.722167,0.800000,0.875200
RobBERT,0.599380,146.4,313.6,64.4,39.6,0.866843,0.759692,0.737903,0.815603,0.990318


In [25]:
'''
torch.load(model1, 'model1.pt')
model1.train_model(train_df1)
result1, model_outputs1, wrong_predictions1 = model1.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
print(result1)
del model1
gc.collect()
'''
#gc.collect()
#torch.cuda.empty_cache()
limitsave=0

'''
train_df1 = pd.DataFrame({ 'text_cleaned': X1_train, 'target': y1_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X1_test, 'target': y1_test })


train_df1 = pd.DataFrame({ 'text_cleaned': X2_train, 'target': y2_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X2_test, 'target': y2_test })

train_df1 = pd.DataFrame({ 'text_cleaned': X3_train, 'target': y3_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X3_test, 'target': y3_test })
'''
train_df1 = pd.DataFrame({ 'text_cleaned': X4_train, 'target': y4_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X4_test, 'target': y4_test })


for i in range(0,N_ITER):
    model1 = ClassificationModel(
    "bert", "bert-base-multilingual-cased",
        use_cuda = True,
        args=train_args
    )
    model1.train_model(train_df1)
    result1, model_outputs1, wrong_predictions1 = model1.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1)
    if(i<limitsave):
        torch.save(model1, 'model1'+str(i)+'.pt')
    del model1
    gc.collect()
    torch.cuda.empty_cache()
    if(i==0):
        dfResultsModels1 = pd.DataFrame.from_dict(result1, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1, orient="index").T
        dfResultsModels1 = dfResultsModels1.append(dfResultsModels1b)
        
dfResultsModels1Trans = pd.DataFrame(dfResultsModels1.mean(axis=0)).T
dfResultsModels1Trans['f1'] = calcule_f1(dfResultsModels1Trans)

dfResultsModelsTotal = dfResultsModels1Trans.copy()
dfResultsModelsTotal.to_csv('dfResultsModelsTotal-4.csv')

#dfResultsModelsTotal = pd.read_csv('dfResultsModelsTotal.csv', encoding='utf8', error_bad_lines=False)
        
for i in range(0,N_ITER):
    model1c = ClassificationModel(
    "roberta", "roberta-base",
        use_cuda = True,
        args=train_args
    )
    model1c.train_model(train_df1)

    result1c, model_outputs1c, wrong_predictions1c = model1c.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1c)
    if(i<limitsave):
        torch.save(model1c, 'model1c'+str(i)+'.pt')
    del model1c
    gc.collect()
    if(i==0):
        dfResultsModels1c = pd.DataFrame.from_dict(result1c, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1c, orient="index").T
        dfResultsModels1c = dfResultsModels1c.append(dfResultsModels1b)

dfResultsModels1cTrans = pd.DataFrame(dfResultsModels1c.mean(axis=0)).T
dfResultsModels1cTrans['f1'] = calcule_f1(dfResultsModels1cTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1cTrans)
dfResultsModelsTotal.to_csv('dfResultsModelsTotal-4.csv')



for i in range(0,N_ITER):
    model1d = ClassificationModel(
    "distilbert", "distilbert-base-cased",
        use_cuda = True,
        args=train_args
    )
    model1d.train_model(train_df1)

    result1d, model_outputs1d, wrong_predictions1d = model1d.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1d)
    if(i<limitsave):
        torch.save(model1d, 'model1d'+str(i)+'.pt')
    del model1d
    gc.collect()
    if(i==0):
        dfResultsModels1d = pd.DataFrame.from_dict(result1d, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1d, orient="index").T
        dfResultsModels1d = dfResultsModels1d.append(dfResultsModels1b)

dfResultsModels1dTrans = pd.DataFrame(dfResultsModels1d.mean(axis=0)).T
dfResultsModels1dTrans['f1'] = calcule_f1(dfResultsModels1dTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1dTrans)

dfResultsModelsTotal.to_csv('dfResultsModelsTotal-4.csv')



for i in range(0,N_ITER):
    model1e = ClassificationModel(
    "camembert", "camembert-base",
        use_cuda = True,
        args=train_args
    )
    model1e.train_model(train_df1)

    result1e, model_outputs1e, wrong_predictions1e = model1e.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1e)
    if(i<limitsave):
        torch.save(model1e, 'model1e'+str(i)+'.pt')
    del model1e
    gc.collect()
    if(i==0):
        dfResultsModels1e = pd.DataFrame.from_dict(result1e, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1e, orient="index").T
        dfResultsModels1e = dfResultsModels1e.append(dfResultsModels1b)

dfResultsModels1eTrans = pd.DataFrame(dfResultsModels1e.mean(axis=0)).T
dfResultsModels1eTrans['f1'] = calcule_f1(dfResultsModels1eTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1eTrans)

dfResultsModelsTotal.to_csv('dfResultsModelsTotal-4.csv')


for i in range(0,N_ITER):
    model1f = ClassificationModel(
    "albert", "albert-base-v1",
        use_cuda = True,
        args=train_args
    )
    model1f.train_model(train_df1)

    result1f, model_outputs1f, wrong_predictions1f = model1f.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1f)
    if(i<limitsave):
        torch.save(model1f, 'model1f'+str(i)+'.pt')
    del model1f
    gc.collect()
    if(i==0):
        dfResultsModels1f = pd.DataFrame.from_dict(result1f, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1f, orient="index").T
        dfResultsModels1f = dfResultsModels1f.append(dfResultsModels1b)

dfResultsModels1fTrans = pd.DataFrame(dfResultsModels1f.mean(axis=0)).T
dfResultsModels1fTrans['f1'] = calcule_f1(dfResultsModels1fTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1fTrans)
dfResultsModelsTotal.to_csv('dfResultsModelsTotal-4.csv')



for i in range(0,N_ITER):
    model1g = ClassificationModel(
    "flaubert","flaubert/flaubert_base_cased",
        use_cuda = True,
        args=train_args
    )
    model1g.train_model(train_df1)

    result1g, model_outputs1g, wrong_predictions1g = model1g.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1g)
    if(i<limitsave):
        torch.save(model1g, 'model1g'+str(i)+'.pt')
    del model1g
    gc.collect()
    if(i==0):
        dfResultsModels1g = pd.DataFrame.from_dict(result1g, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1g, orient="index").T
        dfResultsModels1g = dfResultsModels1g.append(dfResultsModels1b)

dfResultsModels1gTrans = pd.DataFrame(dfResultsModels1g.mean(axis=0)).T
dfResultsModels1gTrans['f1'] = calcule_f1(dfResultsModels1gTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1gTrans)
dfResultsModelsTotal.to_csv('dfResultsModelsTotal-4.csv')

for i in range(0,N_ITER):
    model1h = ClassificationModel(
    "roberta","pdelobelle/robbert-v2-dutch-base",
        use_cuda = True,
        args=train_args
    )
    model1h.train_model(train_df1)

    result1h, model_outputs1h, wrong_predictions1h = model1h.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1h)
    if(i<limitsave):
        torch.save(model1h, 'model1h'+str(i)+'.pt')
    del model1h
    gc.collect()
    if(i==0):
        dfResultsModels1h = pd.DataFrame.from_dict(result1h, orient="index").T
    else:
        dfResultsModels1b = pd.DataFrame.from_dict(result1h, orient="index").T
        dfResultsModels1h = dfResultsModels1h.append(dfResultsModels1b)
        
dfResultsModels1hTrans = pd.DataFrame(dfResultsModels1h.mean(axis=0)).T
dfResultsModels1hTrans['f1'] = calcule_f1(dfResultsModels1hTrans)
dfResultsModelsTotal = dfResultsModelsTotal.append(dfResultsModels1hTrans)
dfResultsModelsTotal.to_csv('dfResultsModelsTotal-4.csv')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

  0%|          | 0/1313 [00:00<?, ?it/s]

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

...

Running Epoch 14 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

C:\Users\MICROSOFT\Anaconda3\envs\ED\lib\site-packages\simpletransformers\classification\classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/564 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/18 [00:00<?, ?it/s]

{'mcc': 0.7863255373263358, 'tp': 102, 'tn': 421, 'fp': 22, 'fn': 19, 'auroc': 0.9486036229315523, 'auprc': 0.8655353547365977, 'f1': 0.9273049645390071, 'acc': 0.9273049645390071, 'eval_loss': 0.390689542517066}


In [26]:

#dfResultsModelsTotal.reindex(indexBERT)
dfResultsModelsTotal = dfResultsModelsTotal.reset_index(drop=True)
dfResultsModelsTotal.index = indexBERT
dfResultsModelsTotal

dfResultssModelsTotal4 = dfResultsModelsTotal.copy()

dfResultssModelsTotal4

,mcc,tp,tn,fp,fn,auroc,auprc,f1,acc,eval_loss
BERT,0.822284,102.2,428.4,14.6,18.8,0.950555,0.875754,0.859546,0.940780,0.348302
RoBERTa,0.827651,103.8,427.6,15.4,17.2,0.938974,0.879617,0.864280,0.942199,0.339820
DistilBERT,0.799950,101.0,425.2,17.8,20.0,0.948936,0.873555,0.842369,0.932979,0.367044
CamemBERT,0.776524,103.0,417.4,25.6,18.0,0.952674,0.864695,0.825321,0.922695,0.306505
Albert,0.788981,98.8,425.6,17.4,22.2,0.937977,0.865064,0.833052,0.929787,0.376959
Flaubert,0.789385,103.8,419.0,24.0,17.2,0.930052,0.839554,0.834405,0.926950,0.429935
RobBERT,0.782960,102.0,420.2,22.8,19.0,0.943126,0.862188,0.829943,0.925887,0.403184


In [ ]:
import sys,gc,torch
#sys.getsizeof(wrong_predictions4g)
gc.collect()
torch.cuda.empty_cache()

In [ ]:
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')

result1, model_outputs1, wrong_predictions1 = model1.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
result1b, model_outputs1b, wrong_predictions1b = model1b.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
result1c, model_outputs1c, wrong_predictions1c = model1c.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
result1d, model_outputs1d, wrong_predictions1d = model1d.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
result1e, model_outputs1e, wrong_predictions1e = model1e.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
result1f, model_outputs1f, wrong_predictions1f = model1f.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
result1g, model_outputs1g, wrong_predictions1g = model1g.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
#result1h, model_outputs1h, wrong_predictions1h = model1h.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)


#result2, model_outputs2, wrong_predictions2 = model2.eval_model(test_df2, f1=f1_multiclass, acc=accuracy_score)
#result3, model_outputs3, wrong_predictions3 = model3.eval_model(test_df3, f1=f1_multiclass, acc=accuracy_score)
#result4, model_outputs4, wrong_predictions4 = model4.eval_model(test_df4, f1=f1_multiclass, acc=accuracy_score)


print(result1)
print(result1b)
print(result1c)
print(result1d)
print(result1e)
print(result1f)
print(result1g)
print(result1h)
#print(result2)
#print(result3)
#print(result4)

In [ ]:
import torch
import gc
gc.collect()
print(torch.cuda.memory_summary(device=0, abbreviated=False))


In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
import bert
FullTokenizer = bert.bert_tokenization.FullTokenizer
from tensorflow.keras.models import Model       # Keras is the new high level API for TensorFlow
import math



In [ ]:
max_seq_length = 128  # Your choice here.
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])


# See BERT paper: https://arxiv.org/pdf/1810.04805.pdf
# And BERT implementation convert_single_example() at https://github.com/google-research/bert/blob/master/run_classifier.py

def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [ ]:
import gc
del variables
gc.collect()
print(torch.cuda.memory_summary(device=0, abbreviated=False))
